In [78]:
import sys
sys.path.append("../")
from utils import solve_quadratic# 最適ダイヤ設計コードのテスト

class PathPlanner:
    def __init__(self, **kwargs):
        self.car_spec = kwargs.get("car_spec")
        self.initial_params = kwargs.get("initial_params")
        self.ideal_params_at_end = kwargs.get("ideal_params_at_end")
        self.COURSE_LENGTH = kwargs.get("COURSE_LENGTH")
        self.time_to_exit = self.ideal_params_at_end["ideal_arrive_time"] - self.initial_params["time"]
        
    def decide_is_delayed(self):
        v_0 = self.initial_params["speed"]
        v_lim = self.car_spec["v_max"]
        v_exit = self.ideal_params_at_end["ideal_speed"]
        a_max = self.car_spec["max_acc"]
        a_dec = self.car_spec["max_dec"]
        t_0 = self.initial_params["time"]
        t_end = self.ideal_params_at_end["ideal_arrive_time"]

        if v_0 > v_lim or v_exit > v_lim:
            ValueError("Enter Speedが最高速度を上回っています")
            
        if v_0 > v_exit:
            ValueError("Enter Speedが出口速度を上回っています")

        # 早く着きすぎたか、遅れたか
        is_delayed = self.time_to_exit * v_0 < self.COURSE_LENGTH
        
        return is_delayed

    def describe_params(self):
        v_0 = self.initial_params["speed"]
        v_lim = self.car_spec["v_max"]
        v_exit = self.ideal_params_at_end["ideal_speed"]
        a_max = self.car_spec["max_acc"]
        a_dec = self.car_spec["max_dec"]
        t_0 = self.initial_params["time"]
        t_end = self.ideal_params_at_end["ideal_arrive_time"]
        params_dict = {"v_0": v_0,  "v_lim": v_lim, "v_exit":v_exit, \
                       "a_max": a_max,  "a_dec": a_dec, "t_0":t_0, "t_end":t_end, "length":self.COURSE_LENGTH }
        return params_dict

    # 限られ時間と距離でそもそもVeまで到達するかを判定
    # {isPossible: Bool,  reason: string}を返す. 
    def can_reach_v_exit(self, params):
        delta_v = params["v_exit"] - params["v_0"]
        a_max = params["a_max"]
        time_limit = params["t_end"] - params["t_0"]
        length_limit = params["length"]
        
        if delta_v > a_max * time_limit:
            return {"is_possible":False, "Reason":"Time limit"}
            
        cover_distance = (params["v_exit"]**2 - params["v_0"]**2 ) / a_max * 0.5
        if cover_distance > length_limit:
            return {"is_possible":False, "Reason":"Length limit"}

        return {"is_possible":True, "Reason":"OK"}
        

    """
    Path Planning用の関数

    Input: priority = "speed" | "distance"  ※distanceは未実装
    """
    def solve_path(self, priority="speed"):
        
        if priority != "speed":
            ValueError("Priorityの値が不適切です")

        # priority = "speed"の場合はまずはそもそもVeで帰れるかを判定する。
        can_reach_v_exit_output = self.can_reach_v_exit(params_dict)

        if not can_reach_v_exit_output["is_possible"]:
            

        
        
        # 基本的には [ Acc, Dec, Const ]からなる3区間で構成されるはず
        acceleration_pattern_list = [
            ["Acc", "Const", "Dec"],
            ["Acc", "Dec", "Const"],
            ["Const", "Acc", "Dec"],
            ["Const", "Dec", "Acc"],
            ["Dec", "Acc", "Const"],
            ["Dec", "Const", "Acc"],
        ]
        # 最初のアクション, 次, その次のアクションの継続時間を m1, m2, m3とする
        m1 = 0
        m2 = 0
        m3 = 0
        m1
            
        


In [ ]:
def calc_controlled_eta(initial_params, ideal_params_at_end, car_spec, L_max ):
    v_0 = initial_params["speed"]
    v_lim = car_spec["v_max"]
    v_exit = ideal_params_at_end["ideal_speed"]
    a_max = car_spec["max_acc"]
    a_dec = car_spec["max_dec"]
    t_0 = initial_params["time"]
    t_end = ideal_params_at_end["ideal_arrive_time"]
    L_max = L_max

    if v_0 > v_lim or v_exit > v_lim:
        ValueError("Enter Speedが最高速度を上回っています")

    if v_0 > v_exit:
        ValueError("Enter Speedが出口速度を上回っています")

    if (t_end - t_0) * a_max + v_0 < v_exit:
        print("不可能: 全力で加速するのみ")
        return False

    if (t_end - t_0) * v_0 >= L_max: # Trueだった場合、早く着きすぎているということ
        print("Possible: 早く着きすぎた")
        return True
        
    # ここより上の議論は(v_0, v_exit) と (t_0, t_end) だけで完結する話

    
    # 以下、距離まで織り込んだ議論 
    # 終点で速度をv_exitにするという条件のもとで頑張る
    t_margin = t_end - t_0
    time_to_reach_v_exit = (v_exit - v_0) / a_max
    v_max_possible = (t_margin - time_to_reach_v_exit) / (a_max + a_dec) * a_dec * a_max + v_exit

    print("可能的最高速度:", v_max_possible)

    if v_max_possible > v_lim:
        # この場合は一定速度 v_lim で走る区間がある、s1+s2+s3 > LならOKのやつ
        print("a")
        s_1 = (v_lim**2 - v_0 ** 2) / 2 / a_max
        s_3 = (v_lim**2 - v_exit ** 2) / 2 / a_dec
        t_1 = (v_lim - v_0) / a_max
        t_3 = (v_lim - v_exit) / a_dec
        t_2 = t_end - t_0 - t_1 - t_3
        S = s_1 + s_3 + t_2 * v_lim

        print("可能的最高速度:", v_lim)
        print("t_2=",t_2)
        print("t_3=",t_3)
        print("カバー距離:", S)
        print("達成可能性:", (t_2 < 0 or S > L_max))

        return (t_2 < 0 or S > L_max)

    # これより下は v_lim <= v_maxの場合。要するに行けるとこまで加速する
    # v_0で入ってからv_exitで抜けるまでの総走行距離を計算する
    print("定速区間なし")
    
    S = (v_max**2 - v_0**2) / 2 / a_max + (v_max**2 - v_0**2) / 2 / a_dec
    total_time = (v_max - v_0)/a_max + (v_max - v_exit) / a_dec
    can_cover_the_distance = S > L_max
    is_in_time = total_time < t_margin
    print("カバー距離:", S)
    print(can_cover_the_distance, is_in_time)
    print("達成可能性:", ( is_in_time and can_cover_the_distance))

    return (is_in_time and can_cover_distance)
    

In [73]:
# initial_params
initial_time = 0
v_0=30
initial_params = {"time": initial_time, "speed":v_0}

# ideal_params
v_exit = 30
ideal_arrival_time = 30
ideal_params_at_end = {"ideal_arrive_time": ideal_arrival_time, "ideal_speed": v_exit}

# car_spec
max_acc = 0.5
max_dec = 0.4
v_max = 45
car_spec = {"max_acc":max_acc, "max_dec":max_dec, "v_max":v_max}

#その他の設計
L_max = 1000

calc_controlled_eta(initial_params, ideal_params_at_end, car_spec, L_max)


可能的最高速度: 36.66666666666667
定速区間なし
カバー距離: 2531.25
True False
達成可能性: False


False